In [2]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py

from ipywidgets import widgets 
from IPython.display import display, clear_output, Image
from plotly.graph_objs import *
from plotly.widgets import GraphWidget


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1) 

In [4]:
df['carrier'].unique()


array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [5]:
month = widgets.FloatSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update = False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month] )

textbox = widgets.Text(
    description='Airline:   ',
    value='Delta',
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


### Assign an emptry graph widget with two traces

g = GraphWidget('https://plot.ly/~kevintest/1192/')

In [6]:
def validate():
    # print "validating"
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False

def response(change):
    if validate():
        # print "using date"
        if use_date.value:
            filter_list = [i and j and k for i,j,k in 
                           zip(df['month'] == month.value, df['carrier'] == textbox.value, 
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]
            x1 = temp_df['arr_delay']
            x2 = temp_df['dep_delay']
            g.restyle({'x': [x1], 'name': 'Arrival Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=0) 
            g.restyle({'x': [x2], 'name': 'Departure Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=1)
            g.relayout({'barmode': 'overlay', 'xaxis': {'title': 'Delay in Minutes'},
                        'yaxis': {'title': 'Number of Delays'}})
            # print temp_df.head()
            
        else:
            # print "not using date"
            filter_list = [i and j for i,j in 
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
            x1 = temp_df['arr_delay']
            x2 = temp_df['dep_delay']
            g.restyle({'x': [x1], 'name': 'Arrival Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=0) 
            g.restyle({'x': [x2], 'name': 'Departure Delays', 'type': 'histogram', 
                       'opacity': 0.75}, indices=1)
            g.relayout({'barmode': 'overlay'})
            # print temp_df.head()
    

origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [7]:
display(container)
container2 = widgets.HBox([origin, textbox])
display(container2)
display(g)

In [ ]:
|